# V2 de RICK

In [1]:
import os
import numpy as np
import shutil
import matplotlib.pyplot as plt
from vozyaudio import lee_audio, espectro as calcular_espectro
import subprocess
from scipy.signal import firwin, lfilter, get_window
from scipy.fft import rfft, rfftfreq


In [20]:
AUDIO_PATH = "audios\music.wav"
CARPETA_FRAMES  = 'fotogramas'
FPS = 60
COLOR = "color" # gris, color
MODE = "ampl"    # ampl, fir, fft
# Eliminar y crear carpeta de fotogramas
if os.path.exists(CARPETA_FRAMES):
    shutil.rmtree(CARPETA_FRAMES)
os.makedirs(CARPETA_FRAMES)

In [3]:
# Leer audio
fs, audio = lee_audio(AUDIO_PATH)
audio = audio / np.max(np.abs(audio))  # Normalización

In [4]:
# Filtros predefinidos (definidos fuera para no recalcular en cada llamada)
numcoef = 101
b_low = firwin(numcoef, cutoff=2000, fs=fs)
b_band = firwin(numcoef, [2000, 6000], pass_zero=False, fs=fs)
b_high = firwin(numcoef, cutoff=6000, fs=fs, pass_zero=False)

# Codificador

In [33]:
def generar_frame_cabecera(N, salto, fps, carpeta=CARPETA_FRAMES):
    """
    Codifica N, salto y fps en las primeras 5 filas de un array N×N,
    y lo guarda *sin* normalización ni colormap como PNG.
    """
    header = np.zeros((N, N), dtype=np.uint8)
    hiN, loN   = (N >> 8) & 0xFF, N & 0xFF
    hiS, loS   = (salto >> 8) & 0xFF, salto & 0xFF
    b_fps      = fps & 0xFF

    header[0, :] = hiN
    header[1, :] = loN
    header[2, :] = hiS
    header[3, :] = loS
    header[4, :] = b_fps

    os.makedirs(carpeta, exist_ok=True)
    ruta = os.path.join(carpeta, "frame_0000.png")
    # ¡aquí el cambio clave!
    imageio.imwrite(ruta, header)

In [ ]:
def generarFrame_AudioGeneral():
    """
    Cada fila del frame es una ventana distinta de audio.
    El frame final será de tamaño (n_bloques, N)
    """
    frame_data = []

    for i in range(n_bloques):
        print(f"\rProcesando bloque {i+1}/{n_bloques}", end='', flush=True)

        # Extraer y enventanar bloque
        start = i * salto
        bloque = audio[start : start + N] * ventana

        # Normalizar bloque individualmente a [0, 255]
        bloque_norm = (bloque - np.min(bloque)) / (np.max(bloque) - np.min(bloque) + 1e-12)
        fila = (bloque_norm * 255).astype(np.uint8)

        frame_data.append(fila)

    # Convertir a imagen: cada fila es un bloque
    frame_img = np.vstack(frame_data)

    # Guardar una única imagen
    plt.imsave(f"{CARPETA_FRAMES}/frame_total.png", frame_img, cmap='gray')
    print("\n✅ Frame generado.")


In [6]:
def colorearFila(bloque, modo='amplitud'):
    """
    Devuelve una fila RGB (shape 1×N×3, uint8) coloreada según:
      - modo='amplitud': mapa basado en amplitud temporal
      - modo='fft'      : mapa basado en energía por bin de la FFT
    """
    # Normalización ventana
    bloque = bloque.astype(np.float32)
    if modo == 'ampl':
        amp = np.abs(bloque)
        amp = (amp - amp.min()) / (amp.max() - amp.min() + 1e-12)
        r = 1 - amp
        g = amp
        b = 0.5 * np.ones_like(amp)

    elif modo == 'fft':
        # FFT
        X = np.abs(rfft(bloque, n=N))
        f = rfftfreq(N, 1/fs)
        Xn = (X - X.min())/(X.max() - X.min() + 1e-12)

        # mascarillas
        low_mask  = (f <= 500)
        band_mask = (f > 500) & (f < 5000)
        high_mask = (f >= 5000)

        Rf = Xn * low_mask
        Bf = Xn * band_mask
        Gf = Xn * high_mask

        # interpolación a N columnas
        bins   = np.arange(len(Xn))
        target = np.linspace(0, len(Xn)-1, N)
        interp = lambda arr: np.interp(target, bins, arr)

        r = interp(Rf)
        g = interp(Gf)
        b = interp(Bf)
    elif modo == 'fir':
        # filtrar en el dominio temporal y colorear por muestra
        y_low  = lfilter(b_low,  1.0, bloque)
        y_band = lfilter(b_band, 1.0, bloque)
        y_high = lfilter(b_high, 1.0, bloque)

        E_l = np.abs(y_low)
        E_b = np.abs(y_band)
        E_h = np.abs(y_high)
        # normaliza cada uno 0..1
        r = (E_l - E_l.min())/(E_l.max()-E_l.min()+1e-12)
        g = (E_h - E_h.min())/(E_h.max()-E_h.min()+1e-12)
        b = (E_b - E_b.min())/(E_b.max()-E_b.min()+1e-12)

    else:
        raise ValueError("modo debe ser 'amplitud', 'fft' o 'fir'")

    # stack y convertir a uint8
    fila = np.stack([r, g, b], axis=1)[np.newaxis, ...]  # (1×N×3)
    return (fila * 255).astype(np.uint8)


In [7]:
def generarFrames_VentanaFrame(color="gris", modo="amplitud", window_type="hann",salto = None ):
    """
    Genera frames donde cada fila del frame representa un bloque de audio.
    
    Parámetros:
    - color: "gris" o "color"
    - modo: para el coloreado, "amplitud", "fft" o "fir"
    - window_type: nombre de la ventana (p. ej. "hann", "hamming", "boxcar", "blackman", etc.)
    
    """

    # 1) calculamos N a partir de FPS
    N = fs // FPS
    if N % 2 != 0:
        N += 1

    # 2) elegimos el salto
    if salto is None:
        salto = N // 2

    print(f"Tamaño de ventana N: {N}, salto: {salto}, resolución: {N}x{N}")

    # 3) ventana de análisis
    ventana = get_window(window_type, N)

    # 4) número total de bloques
    n_bloques = (len(audio) - N) // salto
    print(f"Numero de bloques {n_bloques}, FPS {FPS}")

    # Frame 0 - metadatos
    generar_frame_cabecera(N, salto, FPS, CARPETA_FRAMES)

    for i in range(n_bloques):
        print(f"\rProcesando bloque {i+1}/{n_bloques}", end='', flush=True)

        start = i * salto
        bloque = audio[start : start + N] * ventana

        if color == "gris":
            bloque_norm = (bloque - np.min(bloque)) / (np.max(bloque) - np.min(bloque) + 1e-12)
            fila = (bloque_norm * 255).astype(np.uint8)
            frame_img = np.tile(fila, (N, 1))
            plt.imsave(f"{CARPETA_FRAMES}/frame_{i + 1:04d}.png", frame_img, cmap='gray')

        elif color == "color":
            fila = colorearFila(bloque, modo)  # ampl , fft , FIR
            frame_img = np.tile(fila, (N, 1, 1))
            plt.imsave(f"{CARPETA_FRAMES}/frame_{i+1:04d}.png", frame_img)

        else:
            raise ValueError("Modo no reconocido. Usa 'gris' o 'color'.")


In [34]:
generarFrames_VentanaFrame(COLOR,MODE)

Tamaño de ventana N: 736, salto: 368, resolución: 736x736
Numero de bloques 1196, FPS 60
Procesando bloque 1196/1196

In [35]:
# 1) Carpeta export y rutas
os.makedirs("exports", exist_ok=True)
audio_base   = os.path.splitext(os.path.basename(AUDIO_PATH))[0]
output_name  = f"{audio_base}_{MODE}_{COLOR}.mp4"
output_path  = os.path.join("exports", output_name)

# 2) Borra antiguo
if os.path.exists(output_path):
    os.remove(output_path)

# 3) Ejecuta el batch desde Jupyter, expandiendo las variables
!generarVideo2.bat {FPS} "{output_path}"


c:\Users\luigi\Documents\UPV\3ro\Voz\PixelSounds>ffmpeg -framerate 60 -i fotogramas/frame_%04d.png -c:v libx264 -crf 0 -preset veryslow -pix_fmt yuv444p exports\music_ampl_color.mp4 


ffmpeg version 7.1-essentials_build-www.gyan.dev Copyright (c) 2000-2024 the FFmpeg developers
  built with gcc 14.2.0 (Rev1, Built by MSYS2 project)
  configuration: --enable-gpl --enable-version3 --enable-static --disable-w32threads --disable-autodetect --enable-fontconfig --enable-iconv --enable-gnutls --enable-libxml2 --enable-gmp --enable-bzlib --enable-lzma --enable-zlib --enable-libsrt --enable-libssh --enable-libzmq --enable-avisynth --enable-sdl2 --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxvid --enable-libaom --enable-libopenjpeg --enable-libvpx --enable-mediafoundation --enable-libass --enable-libfreetype --enable-libfribidi --enable-libharfbuzz --enable-libvidstab --enable-libvmaf --enable-libzimg --enable-amf --enable-cuda-llvm --enable-cuvid --enable-dxva2 --enable-d3d11va --enable-d3d12va --enable-ffnvcodec --enable-libvpl --enable-nvdec --enable-nvenc --enable-vaapi --enable-libgme --enable-libopenmpt --enable-libopencore-amrwb --enable-libmp3lame --e

In [ ]:
if os.path.exists(CARPETA_FRAMES):
    shutil.rmtree(CARPETA_FRAMES)
os.makedirs(CARPETA_FRAMES)

# Decodificador

In [48]:
import os
import cv2
import imageio
import numpy as np
from scipy.signal import get_window
from scipy.io import wavfile as wav
from vozyaudio import sonido

In [49]:
def extract_all_frames(video_path, output_dir, prefix="frame_", fmt="png"):
    """
    Extrae todos los frames de un vídeo a una carpeta.
    """
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        raise IOError(f"No se pudo abrir el vídeo {video_path}")
    os.makedirs(output_dir, exist_ok=True)

    idx = 0
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        fname = f"{prefix}{idx:04d}.{fmt}"
        cv2.imwrite(os.path.join(output_dir, fname), frame)
        idx += 1

    cap.release()
    return idx  # número total de frames extraídos

In [50]:
def extraer_metadatos_cabecera_rows(frame):
    """
    Extrae N, salto y fps de un array 2D o 3D codificado por filas:
      fila 0→hiN, 1→loN, 2→hiS, 3→loS, 4→fps.
    """
    # asegurar uint8 y escala de grises
    if frame.dtype != np.uint8:
        frame = np.clip(frame * 255, 0, 255).astype(np.uint8)
    if frame.ndim == 3:
        frame = frame[..., 0]

    hiN  = int(frame[0, 0]);     loN  = int(frame[1, 0])
    hiS  = int(frame[2, 0]);     loS  = int(frame[3, 0])
    fpsb = int(frame[4, 0])

    N     = (hiN << 8) + loN
    salto = (hiS << 8) + loS
    fps   = fpsb

    return N, salto, fps


In [55]:
def reconstruir_audio_de_frames_norm(folder, output_wav, modo="gris", window_type="hann"):
    # 1) leer cabecera
    frame0 = imageio.imread(os.path.join(folder, "frame_0000.png"))
    N, salto, fps = extraer_metadatos_cabecera_rows(frame0)
    fs_recon = int(round(N * fps))

    # 2) listar frames
    files = sorted(f for f in os.listdir(folder)
                   if f.startswith("frame_") and f!="frame_0000.png")

    # 3) buffers para señal y pesos
    L = N + salto*(len(files)-1)
    audio = np.zeros(L, dtype=np.float32)
    pesos = np.zeros(L, dtype=np.float32)
    ventana = get_window(window_type, N)

    # 4) procesar
    for i, fname in enumerate(files):
        img = imageio.imread(os.path.join(folder, fname)).astype(np.float32)/255.0

        # obtener bloque (1D)
        if modo=="gris":
            if img.ndim==3:
                gray = 0.299*img[:,:,0] + 0.587*img[:,:,1] + 0.114*img[:,:,2]
            else:
                gray = img
            bloque = 2*gray[0] - 1
        else:
            bloque = 2*np.mean(img[0], axis=1) - 1

        start = i*salto
        audio[start:start+N] += bloque * ventana
        pesos[start:start+N] += ventana

    # 5) normalizar por la superposición de ventanas
    audio /= (pesos + 1e-12)

    # 6) guardar y reproducir
    wav.write(output_wav, fs_recon, (audio*32767).astype(np.int16))
    print("Guardado WAV en", output_wav, "fs =", fs_recon)
    sonido(audio, fs_recon)
    return audio, fs_recon

In [36]:
# 1) Extraer todos los frames del MP4 a disco
video_path = "exports/music_ampl_color.mp4"
frames_dir = "analisis"
num = extract_all_frames(video_path, frames_dir)
print(f"Extraídos {num} frames en '{frames_dir}'")

Extraídos 1197 frames en 'analisis'


In [58]:
# 2) Reconstruir el audio a partir de esos PNGs
output_wav = "exports/music_fir_color_decoded.wav"

audio_rec, fs_rec = reconstruir_audio_de_frames_norm(
    frames_dir,
    output_wav,
    modo="gris",        # o "color" o gris
    window_type="hann"  # o la ventana que uses
)
print("Audio reconstruido a", output_wav, "con fs =", fs_rec)

C:\Users\luigi\AppData\Local\Temp\ipykernel_12652\72737284.py:3: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  frame0 = imageio.imread(os.path.join(folder, "frame_0000.png"))
C:\Users\luigi\AppData\Local\Temp\ipykernel_12652\72737284.py:19: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  img = imageio.imread(os.path.join(folder, fname)).astype(np.float32)/255.0


Guardado WAV en exports/music_fir_color_decoded.wav fs = 44896


Audio reconstruido a exports/music_fir_color_decoded.wav con fs = 44896


In [37]:
ruta_frame = os.path.join("fotogramas", "frame_0000.png")
frame_orig = imageio.imread(ruta_frame)

# Extrae y muestra los metadatos
N, salto, fps = extraer_metadatos_cabecera_rows(frame_orig)
print(f"Original → N: {N}, salto: {salto}, fps: {fps}")

Original → N: 736, salto: 368, fps: 60


C:\Users\luigi\AppData\Local\Temp\ipykernel_12652\1256309531.py:2: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  frame_orig = imageio.imread(ruta_frame)
